In [185]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 

from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved
import matplotlib.pyplot as plt 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
translator = CirqTranslater(3)
u1_layer_db = u1_layer(translator, block_id=0)
cnots_db = cnot_layer(translator, touching=False, block_id=1)

In [183]:
circuit_db = concatenate_dbs([cnots_db, cnots_db,u1_layer_db])
circuit, circuit_db = translator.give_circuit(circuit_db)

In [263]:
qubits_involved = get_qubits_involved(circuit, circuit_db)

gates_on_qubit = {q:[] for q in qubits_involved}
on_qubit_order = {q:[] for q in qubits_involved}
gate_analyzed = {q:[] for q in qubits_involved}

for order_gate, ind_gate in enumerate( circuit_db["ind"]):
    if ind_gate < translator.number_of_cnots:
        control, target = translator.indexed_cnots[str(ind_gate)]
        gates_on_qubit[control].append(ind_gate)
        gates_on_qubit[target].append(ind_gate)
        on_qubit_order[control].append(order_gate)                
        on_qubit_order[target].append(order_gate)  
        gate_analyzed[control].append(False)
        gate_analyzed[target].append(False)
    else:
        gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
        on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
        gate_analyzed[(ind_gate-translator.n_qubits)%translator.n_qubits].append(False)


In [248]:
simplified_db = circuit_db.copy()

In [249]:
def rule_1(translator, simplified_db, on_qubit_order, qubit_gates_path, gate_analized):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path):
            if ind_gate < translator.number_of_cnots:
                control, target = translator.indexed_cnots[str(ind_gate)]
                if (q == control) and (order_gate_on_qubit == 0):
                    pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                    simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                    gate_analyzed[q][orded_gate_on_qubit] = True
                    simplification = True
                    break
            else:       
                simplification = False
                gate_analyzed[q][orded_gate_on_qubit] = True
    simplified_db = simplified_db.reset_index(drop=True)
    return simplification, simplified_db


In [250]:
simplification, simplified_db = rule_1(translator, simplified_db, on_qubit_order, qubit_gates_path, gate_analyzed)

In [262]:
def rule_2(translator, simplified_db, on_qubit_order, qubit_gates_path, gate_analized):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            next_ind_gate = qubit_gates_path[order_gate_on_qubit+1]
            if (ind_gate < translator.number_of_cnots) and (ind_gate == next_ind_gate):
                #next_control, next_target = translator.indexed_cnots[str(next_ind_gate)]
                
                ## up to here, I know that current gate affecting this qubit is the same than the one coming next
                ## but what about the other qubit ?
                control, target = translator.indexed_cnots[str(ind_gate)]

                not_gates_in_between = False
                this_qubit = q
                other_qubit = [control, target]
                other_qubit = other_qubit.remove(q)[0]
                    
                ## now we need to check what happens in the other_qubit
                for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit][:-1]):
                    if (ind_gate_other == ind_gate) and (gates_on_qubit[other_qubit][qord_other +1] == ind_gate):
                        
                        ## if we append the CNOT for q and other_q on the same call, and also for the consecutive
                        ## note that in between there can be other calls for other qubits
                        order_call_q = on_qubit_order[q][order_gate_on_qubit]
                        order_call_other_q = on_qubit_order[other_qubit][qord_other]
                        
                        order_call_qP1 = on_qubit_order[q][order_gate_on_qubit+1]
                        order_call_other_qP1 = on_qubit_order[other_qubit][qord_other+1]
                        
                        ## then it's kosher to say they are consecutively applied (if only looking at the two qubits)
                        if (order_call_q == order_call_other_q) and (order_call_qP1 == order_call_other_qP1):
                            
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit+1]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)

                            gate_analyzed[q][orded_gate_on_qubit] = True
                            gate_analyzed[q][orded_gate_on_qubit+1] = True
                            simplification = True
                            break
            else:       
                simplification = False
                gate_analyzed[q][orded_gate_on_qubit] = True
    simplified_db = simplified_db.reset_index(drop=True)
    return simplification, simplified_db


In [260]:
simplification, simplified_db = rule_1(translator, simplified_db, on_qubit_order, qubit_gates_path, gate_analyzed)

{0: [0, 1, 3, 4, 6, 7, 8], 1: [0, 2, 3, 5, 9, 10, 11], 2: [12, 13, 14]}

In [258]:
l.remove(2)

In [259]:
l

[1]